In [1]:
#importing libraries
import csv

import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Flatten,Dense,Cropping2D,Activation,Lambda,Dropout

from keras.layers.convolutional import Convolution2D

from sklearn.utils import shuffle

Using TensorFlow backend.


In [2]:


# loading csv data
lines=[]
csv_path="dataset/lap1/driving_log.csv"
with open(csv_path) as csv_file:
    reader=csv.reader(csv_file)
    for line in reader:
        lines.append(line)
        

from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(lines, test_size=0.2)
    

In [3]:
'''
%matplotlib inline
images=[]
measurement=[]
correction=0.25
#loading images and measurement data
for line in lines:
    # for center images
    source_path=line[0]
    image_name=source_path.split('\\')[-1]
    image_name=image_name.split('/')[-1]
    current_path="dataset/lap1/IMG/"+image_name
    #print(current_path)
    image=plt.imread(current_path)
    images.append(image)
    measurement.append(float(line[3]))
    
    #for left images
    source_path=line[1]
    image_name=source_path.split('\\')[-1]
    image_name=image_name.split('/')[-1]
    current_path="dataset/lap1/IMG/"+image_name
    #print(current_path)
    image=plt.imread(current_path)
    images.append(image)
    measurement.append(float(line[3])+correction)
    
    #for right images
    source_path=line[2]
    image_name=source_path.split('\\')[-1]
    image_name=image_name.split('/')[-1]
    current_path="dataset/lap1/IMG/"+image_name
    #print(current_path)
    image=plt.imread(current_path)
    images.append(image)
    measurement.append(float(line[3])-correction)
    print("laoding of image done")
    
'''

'\n%matplotlib inline\nimages=[]\nmeasurement=[]\ncorrection=0.25\n#loading images and measurement data\nfor line in lines:\n    # for center images\n    source_path=line[0]\n    image_name=source_path.split(\'\\\')[-1]\n    image_name=image_name.split(\'/\')[-1]\n    current_path="dataset/lap1/IMG/"+image_name\n    #print(current_path)\n    image=plt.imread(current_path)\n    images.append(image)\n    measurement.append(float(line[3]))\n    \n    #for left images\n    source_path=line[1]\n    image_name=source_path.split(\'\\\')[-1]\n    image_name=image_name.split(\'/\')[-1]\n    current_path="dataset/lap1/IMG/"+image_name\n    #print(current_path)\n    image=plt.imread(current_path)\n    images.append(image)\n    measurement.append(float(line[3])+correction)\n    \n    #for right images\n    source_path=line[2]\n    image_name=source_path.split(\'\\\')[-1]\n    image_name=image_name.split(\'/\')[-1]\n    current_path="dataset/lap1/IMG/"+image_name\n    #print(current_path)\n    imag

In [11]:
%matplotlib inline
images=[]
measurement=[]
correction=0.1

def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            measurement = []
            for batch_sample in batch_samples:
                

                #center lane
                image_name=batch_sample[0].split('\\')[-1]
                image_name=image_name.split('/')[-1]
                current_path="dataset/lap1/IMG/"+image_name
                #print(current_path)
                image=plt.imread(current_path)
                images.append(image)
                measurement.append(float(batch_sample[3]))
                
                #for left images
                
                image_name=batch_sample[1].split('\\')[-1]
                image_name=image_name.split('/')[-1]
                current_path="dataset/lap1/IMG/"+image_name
                #print(current_path)
                image=plt.imread(current_path)
                images.append(image)
                measurement.append(float(batch_sample[3])+correction)
                
                #for right images
               
                image_name=batch_sample[2].split('\\')[-1]
                image_name=image_name.split('/')[-1]
                current_path="dataset/lap1/IMG/"+image_name
                #print(current_path)
                image=plt.imread(current_path)
                images.append(image)
                measurement.append(float(batch_sample[3])-correction)
                
            print("laoding of image done")
            # trim image to only see section with road
            images,angles=augment(images,measurement)
            X_train = np.array(images)
            y_train = np.array(angles)
            yield shuffle(X_train, y_train)


In [12]:
train_generator = generator(train_samples, batch_size=64)
validation_generator = generator(validation_samples, batch_size=64)

In [13]:
#data augmentation

def augment(images,measurement):
    aug_img=[]
    aug_measurement=[]
    for img,measure in zip(images,measurement):
        aug_img.append(img)
        aug_measurement.append(measure)
        aug_img.append(np.fliplr(img))
        aug_measurement.append(measure*-1.0)
    return aug_img,aug_measurement
       

In [14]:
#preprocessing data

def preprocess_images(images):
    cropping2D(cropping=())
    

In [15]:
#X_train=np.array(aug_img)
#y_train=np.array(aug_measurement)




model=Sequential()

model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,10),(0,0))))

#Nvidia model
model.add(Convolution2D(24,5,5,activation=('relu'),subsample=(2,2)))

model.add(Convolution2D(36,5,5,activation=('relu'),subsample=(2,2)))
model.add(Convolution2D(48,5,5,activation=('relu'),subsample=(2,2)))
model.add(Dropout(0.4))
model.add(Convolution2D(64,3,3,activation=('relu')))
model.add(Convolution2D(64,3,3,activation=('relu')))

model.add(Flatten())
model.add(Dense(100))
model.add(Dropout(0.2))
model.add(Dense(50))
model.add(Dense(1))

model.compile(loss='mse',optimizer='adam')
#model.fit(X_train,y_train,validation_split=0.2,shuffle=True,nb_epoch=7,verbose=1,batch_size=128)
model.fit_generator(train_generator, samples_per_epoch=len(train_samples)*6, validation_data=validation_generator,nb_val_samples=len(validation_samples)*6, nb_epoch=6,verbose=1)

model.save("model.h5")



Epoch 1/7
laoding of image done
laoding of image done
laoding of image done
  384/42696 [..............................] - ETA: 153s - loss: 0.0214laoding of image done
laoding of image done
  768/42696 [..............................] - ETA: 117s - loss: 0.0340laoding of image done
laoding of image done
 1536/42696 [>.............................] - ETA: 99s - loss: 0.0267 laoding of image done
laoding of image done
 1920/42696 [>.............................] - ETA: 95s - loss: 0.0252laoding of image done
laoding of image done
 2304/42696 [>.............................] - ETA: 92s - loss: 0.0240laoding of image done
laoding of image done
 3072/42696 [=>............................] - ETA: 88s - loss: 0.0235laoding of image done
laoding of image done
 3456/42696 [=>............................] - ETA: 86s - loss: 0.0232laoding of image done
laoding of image done
 3840/42696 [=>............................] - ETA: 85s - loss: 0.0228laoding of image done
laoding of image done
33024/426

42624/42696 [============================>.] - ETA: 0s - loss: 0.0120laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
42696/42696 [==============================] - 101s - loss: 0.0120 - val_loss: 0.0062
Epoch 2/7
laoding of image done
21504/42696 [==============>...............] - ETA: 43s - loss: 0.0063laoding of image done


42624/42696 [============================>.] - ETA: 0s - loss: 0.0057laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
42696/42696 [==============================] - 100s - loss: 0.0057 - val_loss: 0.0051
Epoch 3/7
laoding of image done
 9984/42696 [======>.......................] - ETA: 66s - loss: 0.0055laoding of image done


42624/42696 [============================>.] - ETA: 0s - loss: 0.0052laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done


laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
42696/42696 [==============================] - 100s - loss: 0.0052 - val_loss: 0.0048
Epoch 4/7
laoding of image done
32640/42696 [=====================>........] - ETA: 20s - loss: 0.0050laoding of image done


42624/42696 [============================>.] - ETA: 0s - loss: 0.0049laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
42696/42696 [==============================] - 100s - loss: 0.0049 - val_loss: 0.0045
Epoch 5/7
laoding of image done
21120/42696 [=============>................] - ETA: 43s - loss: 0.0051laoding of image done


42624/42696 [============================>.] - ETA: 0s - loss: 0.0046laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
42696/42696 [==============================] - 100s - loss: 0.0046 - val_loss: 0.0043
Epoch 6/7
laoding of image done
 9600/42696 [=====>........................] - ETA: 67s - loss: 0.0045laoding of image done


42624/42696 [============================>.] - ETA: 0s - loss: 0.0042laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done


laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
42696/42696 [==============================] - 100s - loss: 0.0042 - val_loss: 0.0043
Epoch 7/7
laoding of image done
32256/42696 [=====================>........] - ETA: 21s - loss: 0.0042laoding of image done


42624/42696 [============================>.] - ETA: 0s - loss: 0.0041laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
laoding of image done
42696/42696 [==============================] - 100s - loss: 0.0041 - val_loss: 0.0039
